# Data exploratory - Initial

### this notebook is in charge of pre-processing transaction data, removing invalid data and splitting them into proper subfile sets

### 0. Initialisation

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from pandas import Series
from sklearn.preprocessing import LabelEncoder
import numpy as np
import modules.file_helper as file_help
import modules.data_helper as data_help

### 1. Load dataset 

In [2]:
#load DVF (2019) from https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres/ if not yet loaded
file_help.initialize_sells_referential()


Data already downloaded


In [3]:
#load data in memory
global_data = pd.read_csv(os.path.join(file_help.DATA_IN_FOLDER, file_help.FILE_NAME_IN), encoding='utf-8', sep='|', decimal=',')

### 2. Exploratory Data Analysis

In [4]:
#basic stats about data
print(global_data.describe().transpose())
# Check the number of data points in the data set
print(f'Nb records {len(global_data)}')
# Check the number of features in the data set
print(f'Nb columns {len(global_data.columns)}')
# Check the data types
print(f'Data types {global_data.dtypes.unique()}')

#count empty columns
tmp_data = global_data.dropna(axis = 1, how ='all') 
print(f'Nb empty columns {len(global_data.columns) - len(tmp_data.columns)}')
del(tmp_data)

#deep analysis
profil = ProfileReport(global_data)

#generate profiling report. If not exists. This can take more than 10 minutes
#set True to force update of file
force = False

if os.path.exists(os.path.join( file_help.DATA_REPORTS_FOLDER ,'rapport_initial.html')) and not force: 
    print('Initial report already generated at ' + os.path.join( file_help.DATA_REPORTS_FOLDER ,'rapport_initial.html'))    
else :
    profil.to_file(output_file= os.path.join( file_help.DATA_REPORTS_FOLDER ,'rapport_initial.html'))

del(profil)

                                count           mean           std      min  \
Reference document                1.0    1110.000000           NaN  1110.00   
2 Articles CGI                    1.0       1.000000           NaN     1.00   
3 Articles CGI                    1.0      80.000000           NaN    80.00   
4 Articles CGI                    1.0     119.000000           NaN   119.00   
No disposition              2535792.0       1.199567  7.401709e+00     1.00   
Valeur fonciere             2506530.0  971597.040251  7.856914e+06     0.01   
No voie                     1510153.0     727.027501  2.076794e+03     1.00   
Code postal                 2507467.0   51976.389759  2.730259e+04  1000.00   
Code commune                2535791.0     208.996898  1.670737e+02     0.00   
Prefixe de section           126834.0     459.487259  3.260986e+02     1.00   
No plan                     2535790.0     403.790974  5.546665e+02     1.00   
Surface Carrez du 2eme lot    54275.0      63.516572

### 3. Global cleanup of the dataset

In [5]:
#clean up data - remove empty columns
curated_data = global_data.dropna(axis = 1, how ='all') 
print(f'New number of column {len(curated_data.columns)}/{len(global_data.columns)}')

#remove duplicated rows
curated_data.drop_duplicates(inplace=True)
print(f'New number of row {len(curated_data)}/{len(global_data)}')


New number of column 42/43
New number of row 2439669/2535792


#### > Create property referential (house, appartment, ground, ...)

In [6]:
#create property referential
global_property_type = curated_data[['Code type local','Type local']]
global_property_type.drop_duplicates(inplace=True)
global_property_type.dropna(inplace=True)

#add 'Autre' property type
global_property_type.loc[len(global_property_type)] = [len(global_property_type)+1,'Autre']

global_property_type.sort_values(by=['Code type local'], inplace=True)
global_property_type.set_index('Code type local')
global_property_type['Code type local'] = global_property_type['Code type local'].astype(int)

global_property_type.to_csv(os.path.join(file_help.DATA_OUT_FOLDER, 'property_type_referential.csv'), index=False, sep=';')


In [7]:
# drop sells with more than one lot
curated_data.drop(curated_data.loc[curated_data["Nombre de lots"]> 1].index, inplace=True)
curated_data.groupby("Nombre de lots")[['Valeur fonciere']].count().sort_values("Nombre de lots")

,Valeur fonciere
Nombre de lots,
0.0,1636095
1.0,615030


In [8]:
#drop unused columns
curated_data.drop(columns=['No disposition','Date mutation','No voie','B/T/Q','Type de voie','Code voie','Voie','Prefixe de section','Section','No plan','No Volume','1er lot','Surface Carrez du 1er lot','2eme lot','Surface Carrez du 2eme lot','3eme lot','Surface Carrez du 3eme lot','4eme lot','Surface Carrez du 4eme lot','5eme lot','Surface Carrez du 5eme lot','Nombre de lots','Commune', 'Type local'], inplace=True, errors='ignore')

#drop rows with empty sales costs
curated_data.dropna(subset = ['Valeur fonciere'], inplace = True) 

print(f'New number of column {len(curated_data.columns)}/{len(global_data.columns)}')
print(f'New number of records {len(curated_data)}/{len(global_data)}')

New number of column 18/43
New number of records 2251126/2535792


#### > from additionnal referential, create our own to get gps coordinates and insee code

In [9]:
#prepare basic file for correspondance between postal code and code INSEE
#it will be easier to find code INSEE from a town name or postal code
if os.path.exists(os.path.join(file_help.DATA_OUT_FOLDER, 'correspondance-code-insee-code-postal_basic.csv')):
    print('file already exists')
else:
    cp = pd.read_csv(os.path.join(file_help.DATA_IN_FOLDER, 'correspondance-code-insee-code-postal.csv'), encoding='utf-8', sep=';', usecols=['Code INSEE','Code Postal','Commune','Département','Région'])
    cp.to_csv(os.path.join(file_help.DATA_OUT_FOLDER, 'correspondance-code-insee-code-postal_basic.csv'), sep=';', index=False)
    

file already exists


In [10]:
if os.path.exists(os.path.join(file_help.DATA_OUT_FOLDER, 'coord_gps_referential.csv')):
 print('gps coordinates referential already exists at '+ os.path.join(file_help.DATA_IN_FOLDER, 'correspondance-code-insee-code-postal.csv'))
else :
    #load insee referential
    cp = pd.read_csv(os.path.join(file_help.DATA_IN_FOLDER, 'correspondance-code-insee-code-postal.csv'), encoding='utf-8', sep=';', usecols=['Code Commune', 'Code Département', 'Code Postal', 'geo_point_2d', 'Code INSEE'])

    # Create two lists for the loop results to be placed
    lat = []
    lon = []

    # For each row in a varible,
    for row in cp['geo_point_2d']:
        lat.append(row.split(',')[0])
        lon.append(row.split(',')[1])
    
    # Create two new columns from lat and lon
    cp['Latitude'] = lat
    cp['Longitude'] = lon

    #split multiple cp in one row to n rows
    df = cp['Code Postal'].str.split('/').apply(Series, 1).stack()
    df.index = df.index.droplevel(-1) # to line up with df's index
    df.name = 'Code Postal'
    del(cp['Code Postal'])
    cp = cp.join(df)
    
    cp.to_csv(os.path.join(file_help.DATA_OUT_FOLDER, 'coord_gps_referential.csv'), sep=';', index=False, columns= ['Code INSEE', 'Code Postal', 'Code Commune', 'Code Département', 'Latitude', 'Longitude'])
    
    del(lat, lon)    
    del(cp)
    del(df)


gps coordinates referential already exists at c:\prairie\projet8-v2\Immothep\data\in\correspondance-code-insee-code-postal.csv


#### > Fill-up missing postal code then add gps coordinates to sells

In [11]:
#load cities referential
cp = pd.read_csv(os.path.join(file_help.DATA_OUT_FOLDER, 'coord_gps_referential.csv'), encoding='utf-8', sep=';')

#force data type for the merge matching
cp['Code Commune'] = cp['Code Commune'].astype(int)
cp['Code Département'] = cp['Code Département'].astype(str)

#one row has no Code commune
curated_data.dropna(subset = ['Code commune'], inplace=True) 
curated_data['Code commune'] = curated_data['Code commune'].astype(int)
curated_data['Code departement'] = curated_data['Code departement'].astype(str)

curated_data = pd.merge(curated_data, cp, how='inner', left_on=['Code commune','Code departement'], right_on = ['Code Commune','Code Département'])

curated_data.drop(columns=['Code INSEE', 'Code postal', 'Code departement', 'Code commune','Code Commune', 'Code Département', 'Code Postal'], inplace=True)


In [12]:
#Clear data without transaction type - they cannot be manipulated because they cannot be identified
counter = len(curated_data)
curated_data.drop(curated_data[(pd.isna(curated_data['Code type local'])) & (pd.isna(curated_data['Nature culture']))].index, inplace=True)
print(f'Cleaned rows : {counter - len(curated_data)}')

#set 'Autre' value to empty 'Code type local'
other_type = int(global_property_type[global_property_type['Type local'] == 'Autre']['Code type local'])

curated_data['Code type local'] = curated_data['Code type local'].fillna(other_type)
print(curated_data['Code type local'])

curated_data.drop(columns=['Nature culture', 'Nature culture speciale'], inplace=True, errors='ignore')

Cleaned rows : 228953
0          1.0
1          1.0
2          1.0
3          5.0
4          5.0
          ... 
2407036    4.0
2407037    4.0
2407038    4.0
2407039    4.0
2407040    2.0
Name: Code type local, Length: 2178088, dtype: float64


In [13]:
#some sells have wrong number of rooms. Fix it
curated_data['surface_group'] = np.nan
#group sells by surfaces
curated_data.loc[curated_data['Code type local'].isin([1, 2]), 'surface_group'] = np.round( (curated_data[curated_data['Code type local'].isin([1, 2])]['Surface reelle bati']/10))*10

#compute average number of rooms for each groups
rooms = pd.DataFrame( columns=['surface_group','nb_rooms','Code type local'])
#only compute average for Homes and appartments
for type_local in range(1,3):
    for surface_group in curated_data[curated_data['Code type local'] == type_local]['surface_group'].unique():
        nb_rooms = data_help.get_room_mean(surface_group, type_local, curated_data)
        rooms = rooms.append({'surface_group': surface_group, 'nb_rooms': nb_rooms, 'Code type local' : type_local},ignore_index=True )

curated_data = pd.merge(curated_data, rooms, how='left', on=['surface_group', 'Code type local'])
curated_data['delta'] = np.nan

#only replace number of room if difference is more than 50% of the initial value
curated_data.loc[curated_data['Code type local'].isin([1, 2]), 'delta'] = (np.abs(curated_data[curated_data['Code type local'].isin([1, 2])]['Nombre pieces principales'] - curated_data[curated_data['Code type local'].isin([1, 2])]['nb_rooms']) > curated_data[curated_data['Code type local'].isin([1, 2])]['Nombre pieces principales'] /2)

curated_data['Nombre pieces principales'] = np.where(curated_data['delta'], curated_data['nb_rooms'], curated_data['Nombre pieces principales'])

#drop computed data
curated_data.drop(columns=['delta', 'nb_rooms', 'surface_group'], inplace=True)


In [14]:
#complete missing data with 0
curated_data[['Surface reelle bati','Nombre pieces principales', 'Surface terrain']] = curated_data[['Surface reelle bati','Nombre pieces principales', 'Surface terrain']].fillna(0)


In [15]:
#force data type
curated_data['Code type local'] = curated_data['Code type local'].astype(int)
curated_data['Surface reelle bati'] = curated_data['Surface reelle bati'].astype(int)
curated_data['Nombre pieces principales'] = curated_data['Nombre pieces principales'].astype(int)
curated_data['Surface terrain'] = curated_data['Surface terrain'].astype(int)
curated_data['Latitude'] = curated_data['Latitude'].astype(float)
curated_data['Longitude'] = curated_data['Longitude'].astype(float)

#reorder columns
curated_data = curated_data[['Nature mutation', 'Code type local', 'Valeur fonciere', 'Latitude', 'Longitude', 'Surface reelle bati', 'Nombre pieces principales', 'Surface terrain']]
 
#save data by property type
for type in global_property_type.values:
    property_by_type = curated_data[curated_data['Code type local'] == type[0]]
    print(type[1] + ' - ' + str(len(property_by_type)))
    filename = str.format(f'{type[1]}_valeursfoncieres.csv')
    property_by_type.to_csv(os.path.join(file_help.DATA_CURATED_FOLDER, filename), sep=';', index=False )
    del(property_by_type)


Maison - 513516
Appartement - 306925
Dépendance - 346028
Local industriel. commercial ou assimilé - 90393
Autre - 921226


### 5. Global cleanup - post analysis

In [16]:
#deep analysis
profil = ProfileReport(curated_data)

#generate profiling report after first cleaning, if not exists. This can take almost 5 minutes
#set True to force update of file
force = False

if os.path.exists(os.path.join( file_help.DATA_REPORTS_FOLDER ,'rapport_final.html')) and not force:
    print('Final report already generated at ' + os.path.join( file_help.DATA_REPORTS_FOLDER ,'rapport_final.html'))   
else :
    profil.to_file(output_file= os.path.join( file_help.DATA_REPORTS_FOLDER ,'rapport_final.html'))

del(profil)

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 13.52it/s]
